In [12]:
import numpy
import urllib
import scipy.optimize
import random
from math import exp
from math import log

def parseData(fname):
  for l in urllib.urlopen(fname):
    yield eval(l)

print "Reading data..."
data = list(parseData("file:///users/YW/Dropbox/UCSD/FA2015/CSE255/HW/hw1/book_descriptions_50000.json"))
print "done"

def inner(x,y):
  return sum([x[i]*y[i] for i in range(len(x))])

def sigmoid(x):
  return 1.0 / (1 + exp(-x))

# NEGATIVE Log-likelihood
def f(theta, X, y, lam):
  loglikelihood = 0
  for i in range(len(X)):
    logit = inner(X[i], theta)
    loglikelihood -= log(1 + exp(-logit))
    if not y[i]:
      loglikelihood -= logit
  for k in range(len(theta)):
    loglikelihood -= lam * theta[k]*theta[k]
 # print "ll =", loglikelihood
  return -loglikelihood

# NEGATIVE Derivative of log-likelihood
def fprime(theta, X, y, lam):
  dl = [0.0]*len(theta)
  for i in range(len(X)):
    # Fill in code for the derivative
    logit = inner(X[i], theta)
    for k in range(len(dl)):
        dl[k] += (1-sigmoid(logit))*X[i][k]
        if not y[i]:
            dl[k] -= X[i][k]
  for k in range(len(dl)):
    dl[k] -= 2*lam*theta[k]
  # Negate the return value since we're doing gradient *ascent*
  return numpy.array([-x for x in dl])

D_child = [d for d in data if "Children's Books" in d['categories']]
D_notchild =\
  [d for d in data if not("Children's Books" in d['categories'])][:len(D_child)]

data = D_child + D_notchild
random.seed(0)
random.shuffle(data)

X = [[1, "child" in d['description'], "magic" in d['description'], "funny" in d['description']] for d in data]
y = ["Children's Books" in d['categories'] for d in data]

X_train = X[:len(X)/2]
X_valid = X[len(X)/2:3*len(X)/4]
X_test = X[3*len(X)/4:]

y_train = y[:len(y)/2]
y_valid = y[len(y)/2:3*len(y)/4]
y_test = y[3*len(y)/4:]

theta,l,info = scipy.optimize.fmin_l_bfgs_b(f, [0]*len(X[0]), fprime, args = (X_train, y_train, 1.0))
print "Final log likelihood =", -l


Reading data...
done
Final log likelihood = -2297.06998075


In [13]:
from sklearn import linear_model
logreg = linear_model.LogisticRegression(fit_intercept=False)

In [14]:
logreg.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=False,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0)

In [15]:
print logreg.coef_
print theta

[[-0.89569608  2.34170254  0.94704375  0.21062554]]
[-0.88852315  2.3260095   0.91260868  0.20626077]


In [16]:
my_train_predictions = numpy.array([(True if inner(d, theta)>=0 else False) for d in X_train])

In [17]:
my_test_predictions = numpy.array([(True if inner(d, theta)>=0 else False) for d in X_test])

In [18]:
my_valid_predictions = numpy.array([(True if inner(d, theta)>=0 else False) for d in X_valid])

In [19]:
train_predictions = logreg.predict(X_train)
test_predictions = logreg.predict(X_test)
valid_predictions = logreg.predict(X_valid)

In [20]:
print 'My Train Accuracy:%.3f, My Test Accuracy:%.3f, My Valid Accuracy:%.3f'%\
(1-numpy.mean(my_train_predictions!=y_train),
1-numpy.mean(my_test_predictions!=y_test),
1-numpy.mean(my_valid_predictions!=y_valid))
print 'Train Accuracy:%.3f, Test Accuracy:%.3f, Valid Accuracy:%.3f'%\
(1-numpy.mean(train_predictions!=y_train),
1-numpy.mean(test_predictions!=y_test),
1-numpy.mean(valid_predictions!=y_valid))

My Train Accuracy:0.749, My Test Accuracy:0.729, My Valid Accuracy:0.749
Train Accuracy:0.749, Test Accuracy:0.729, Valid Accuracy:0.749


In [25]:
print 1-logreg.score(X_train, y_train)
print 1-logreg.score(X_test, y_test)
print 1-logreg.score(X_valid, y_valid)

0.250965250965
0.271235521236
0.250965250965


In [26]:
lambdas = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]
for i in lambdas:
    theta,l,info = scipy.optimize.fmin_l_bfgs_b(f, [0]*len(X[0]), fprime, args = (X_train, y_train, i))
    my_train_predictions = numpy.array([(True if inner(d, theta)>=0 else False) for d in X_train])
    my_test_predictions = numpy.array([(True if inner(d, theta)>=0 else False) for d in X_test])
    my_valid_predictions = numpy.array([(True if inner(d, theta)>=0 else False) for d in X_valid])
    print 'lambda =', i
    print 'My Train Accuracy:%.3f, My Test Accuracy:%.3f, My Valid Accuracy:%.3f'%\
    (1-numpy.mean(my_train_predictions!=y_train),
    1-numpy.mean(my_test_predictions!=y_test),
    1-numpy.mean(my_valid_predictions!=y_valid))
    print "Final log likelihood =", -l

lambda = 0.0001
My Train Accuracy:0.749, My Test Accuracy:0.729, My Valid Accuracy:0.749
Final log likelihood = -2289.84181645
lambda = 0.001
My Train Accuracy:0.749, My Test Accuracy:0.729, My Valid Accuracy:0.749
Final log likelihood = -2289.84846705
lambda = 0.01
My Train Accuracy:0.749, My Test Accuracy:0.729, My Valid Accuracy:0.749
Final log likelihood = -2289.91495803
lambda = 0.1
My Train Accuracy:0.749, My Test Accuracy:0.729, My Valid Accuracy:0.749
Final log likelihood = -2290.57836569
lambda = 1
My Train Accuracy:0.749, My Test Accuracy:0.729, My Valid Accuracy:0.749
Final log likelihood = -2297.06998075
lambda = 10
My Train Accuracy:0.748, My Test Accuracy:0.727, My Valid Accuracy:0.746
Final log likelihood = -2351.87280459
lambda = 100
My Train Accuracy:0.748, My Test Accuracy:0.727, My Valid Accuracy:0.746
Final log likelihood = -2589.99994874
lambda = 1000
My Train Accuracy:0.748, My Test Accuracy:0.727, My Valid Accuracy:0.746
Final log likelihood = -2817.8378195
